# 經濟、貿易相關新聞篩選器


In [92]:
import gdelt
import pandas as pd
import datetime
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 1000

!python --version
%matplotlib inline
gd1 = gdelt.gdelt(version=1)
gd2 = gdelt.gdelt(version=2)


Python 3.6.3 :: Anaconda custom (x86_64)


In [138]:

def gcam_economics_chapter():
#讀取整個GKG gcam 的表格，取出經濟、金融相關的詞庫章節名稱
    _df = pd.read_csv('./GKGTHEMES.csv')
    contain_words = ['econom','financ','trade']
    dimension_list = list(_df[_df['DimensionHumanName'].str.lower()
                              .str.contains('|'.join(contain_words))]['Variable'])
    dictionary_list = list(_df[_df['DictionaryHumanName'].str.lower()
                               .str.contains('|'.join(contain_words))]['Variable'])
    econo_chapters = dimension_list + dictionary_list
    return econo_chapters


def get_econwords_rate(_df):
#在情緒詞庫中的字數/總字數
    result_df = pd.DataFrame()
    
    for i,row in _df.iterrows(): #每筆取出
        gcam_dict = dict( (n,float(v)) for n,v in (gcam.split(':') for gcam in row['GCAM'].split(',') ) )
        match_chapter = set(gcam_dict) & set(gcam_economics_chapter())
        ecowords_sum = sum([gcam_dict[x] for x in match_chapter])
        rate = '{0:.2f}'.format(ecowords_sum / gcam_dict['wc'])
        tone = row['V2Tone']
        result_df = result_df.append({'RATE':float(rate), 'WC':gcam_dict['wc'], 'TONE':tone, 'URL':row['DocumentIdentifier']},ignore_index=True)
    return result_df



def get_econo_df(_df):
    #取得情緒相關詞典編號
    def unlist_TONE(_df):
        for i,t in _df.iterrows():
            tone_cols = ['Tone','Pos','Neg','Polarity','ActivityReferenceDensity','Self_GroupReferenceDensity','ToneWC']
            tone_values = t['TONE'].split(',')
            tone_dict = dict(zip(tone_cols,tone_values))
            _df.loc[i,'Negative'] = tone_dict['Neg']
            _df.loc[i,'Positive'] = tone_dict['Pos']
            _df.loc[i,'Tone'] = tone_dict['Tone']
        del _df['TONE']
        return _df
    gcam_chapters = gcam_economics_chapter()
    #取得每一筆資料中， gcam 欄位有出現情緒相關辭典的資料
    _df = _df[_df['GCAM'].notnull()] 
    target_df = _df[_df['GCAM'].str.contains('|'.join(gcam_chapters))][['DocumentIdentifier','GCAM','V2Tone']]
    result_df = get_econwords_rate(target_df)
    result_df = result_df[result_df['RATE'] >= 0.40]
    
    return unlist_TONE(result_df)

def get_daily_sentiment_info(gkg,date):
    tone_df = get_econo_df(gkg)
    plot_df = tone_df[['WC','Negative','Positive','Tone']].apply(pd.to_numeric, errors='ignore').reset_index()
    del plot_df['index']
    #plot_df[['Tone','Negative','Positive']].plot()
    result_df = plot_df[['Tone','Positive','Negative']].mean()
    result_df['Articles'] = len(plot_df)
    result_df['Date'] = date
    return result_df

In [139]:
%%time
days = 2
start_date = '2017 11 10'
df = pd.DataFrame()
for d in range(0,days):
    dt = datetime.datetime.strptime(start_date,'%Y %m %d') 
    dt = dt + datetime.timedelta(days = d)
    query_date = datetime.datetime.strftime(dt,'%Y %m %d')
    %time gkg = gd2.Search([query_date],table='gkg',coverage=True)

    %time _df = get_daily_sentiment_info(gkg,query_date)
    
    df = df.append(_df , ignore_index=True)



/anaconda/lib/python3.6/site-packages/gdelt/parallel.py:66: UserWarning: GDELT does not have a url for date time 20171110040000
  warnings.warn(message)
/anaconda/lib/python3.6/site-packages/gdelt/parallel.py:99: UserWarning: GDELT did not return data for date time 20171110040000
  warnings.warn(message)


CPU times: user 5.27 s, sys: 18.2 s, total: 23.5 s
Wall time: 4min 54s


KeyboardInterrupt: 